<a href="https://colab.research.google.com/github/khushboo28sahu/PowerLoadForcasting/blob/main/Power_Load_Forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

import datetime

#from pyearth import Earth
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import joblib
from sklearn.decomposition import PCA


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.DataFrame(pd.read_excel("/content/drive/MyDrive/yamnaya LLP Logistics/Data/ProcessData.xlsx"))

In [ ]:
print("-----Data-----\n\n")
print(data)
print("\n\n-----Data Information -----\n\n")
data.info()

-----Data-----


      Holiday  Bridge  Day        Date  Heure    Conso  temperature  variable 2
0           0       0  Sat  01/10/2022  00:00  14889.0            6       60165
1           0       0  Sat  01/10/2022  01:00  14812.0            5       56678
2           0       0  Sat  01/10/2022  02:00  14469.0            5       55780
3           0       0  Sat  01/10/2022  03:00  14670.0            5       53160
4           0       0  Sat  01/10/2022  04:00  14834.0            5       50207
...       ...     ...  ...         ...    ...      ...          ...         ...
1291        0       0  Wed  23/11/2022  19:00      NaN            7       73029
1292        0       0  Wed  23/11/2022  20:00      NaN            7       71601
1293        0       0  Wed  23/11/2022  21:00      NaN            7       67190
1294        0       0  Wed  23/11/2022  22:00      NaN            6       63621
1295        0       0  Wed  23/11/2022  23:00      NaN            6       66689

[1296 rows x 8 columns

*  We have 8 features viz. Holiday, Bridge, Day, Date, Heure, Conso, Temparature, and variable2.

*  Original Data type is int, float and Object.

* Size of the data is 1296 * 8



In [ ]:

data.loc[(data['Day'] == 'Mon'), 'Day'] = 1
data.loc[(data['Day'] == 'Tue'), 'Day'] = 2
data.loc[(data['Day'] == 'Wed'), 'Day'] = 3
data.loc[(data['Day'] == 'Thu'), 'Day'] = 4
data.loc[(data['Day'] == 'Fri'), 'Day'] = 5
data.loc[(data['Day'] == 'Sat'), 'Day'] = 6
data.loc[(data['Day'] == 'Sun'), 'Day'] = 7

data['Day']=data['Day'].astype(str).astype(int)
# Replacing the sting values of day into integer values(0 - 7)

print("\n\n---Null data types---\n\n")
print(data.isna().sum())




---Null data types---


Holiday          0
Bridge           0
Day              0
Date             0
Heure            0
Conso          168
temperature      0
variable 2       0
dtype: int64


In [ ]:
# hours are converted to integers to help later calculations
data['Heure'] = data['Heure'].apply(lambda x: int(x[:2]))

In [ ]:
data['Heure'].head(24)

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
Name: Heure, dtype: int64

In [ ]:
# Data Creation, X = Features , Y = Label
Y = data['Conso']
X = data.drop(['Conso'], axis = 1)

* X contains all the 7 features
* Y contains label(viz. Conso)





In [ ]:
print("--- Features---\n\n")
print(X)
print("\n\n----X Information ----\n\n")
print(X.info())
print("---Labels---\n\n")
print(Y)
print("---Labels Null Value Information----\n\n")
print(Y.isnull().sum())

--- Features---


      Holiday  Bridge  Day        Date  Heure  temperature  variable 2
0           0       0    6  01/10/2022      0            6       60165
1           0       0    6  01/10/2022      1            5       56678
2           0       0    6  01/10/2022      2            5       55780
3           0       0    6  01/10/2022      3            5       53160
4           0       0    6  01/10/2022      4            5       50207
...       ...     ...  ...         ...    ...          ...         ...
1291        0       0    3  23/11/2022     19            7       73029
1292        0       0    3  23/11/2022     20            7       71601
1293        0       0    3  23/11/2022     21            7       67190
1294        0       0    3  23/11/2022     22            6       63621
1295        0       0    3  23/11/2022     23            6       66689

[1296 rows x 7 columns]


----X Information ----


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296 entries, 0 to 1295
Data

# Train_Test Split(Manually via available conso)

* X contains features and splitted with respect to the available Conso value's
  * X_known_conso contains all the features those have there respective conso values.
  * X_unknown_conso contains all those features which have no conso values.

* Y is a label and splitted with respect to availablity of conso values
  * Y_known_conso contains all the available conso values.
  * Y_unknown_conso contains all the unavailable conso values.

In [ ]:
# Get Index Value for the last avilable conso values
last_available_date_for_conso=data["Conso"].last_valid_index()
print("Last Available date for conso : {}\n\n".format(last_available_date_for_conso))

# setting up the TRAIN TEST UNKNOWN split

# testperiod is a last 1 week data with available conso values
testperiod = 7*24


# ---------------  X W/ Conso and W/O Conso ----------------------
x_known_conso=X[X.index <= last_available_date_for_conso]
print("X with available Conso Value:\n\n {}\n\n".format(x_known_conso))

x_unknown_conso=X.iloc[len(x_known_conso):len(X)]
print("X with unknown available Conso Values :\n\n {} \n\n".format(x_unknown_conso))

# ----------------- Y W/ Conso andn W/O Conso --------------------
y_known_conso = Y[Y.index <= last_available_date_for_conso]
print("Y with known value of Conso :\n\n {}\n\n".format(y_known_conso))

y_unknown_conso = Y.iloc[len(y_known_conso):len(Y)]
print("Y with unknown value of Conso :\n\n{}\n\n".format(y_unknown_conso))


Last Available date for conso : 1127


X with available Conso Value:

       Holiday  Bridge  Day        Date  Heure  temperature  variable 2
0           0       0    6  01/10/2022      0            6       60165
1           0       0    6  01/10/2022      1            5       56678
2           0       0    6  01/10/2022      2            5       55780
3           0       0    6  01/10/2022      3            5       53160
4           0       0    6  01/10/2022      4            5       50207
...       ...     ...  ...         ...    ...          ...         ...
1123        0       0    3  16/11/2022     19            1       80676
1124        0       0    3  16/11/2022     20            0       79188
1125        0       0    3  16/11/2022     21           -1       74911
1126        0       0    3  16/11/2022     22           -1       71606
1127        0       0    3  16/11/2022     23           -2       75092

[1128 rows x 7 columns]


X with unknown available Conso Values :

       Ho

In [ ]:
#------------------ option 1 last testperiod  as test, option 2 is first testperiod  as test
option=2

if(option==1):
  #------------Data with available Conso Values (x_known)
  x_train = x_known_conso.iloc[0:len(x_known_conso)-testperiod]
  y_train = y_known_conso.iloc[0:len(y_known_conso)-testperiod]
  x_train['Date'] = pd.to_datetime(x_train['Date'], format='%d/%m/%Y')
  x_train['Date'] = pd.to_datetime(x_train['Date']).astype(int)/100000000000


  x_test = x_known_conso.iloc[len(x_known_conso)-testperiod:len(x_known_conso)]
  y_test = y_known_conso.iloc[len(y_known_conso)-testperiod:len(y_known_conso)]
  x_test['Date'] = pd.to_datetime(x_test['Date'], format='%d/%m/%Y')
  x_test['Date'] = pd.to_datetime(x_test['Date']).astype(int)/100000000000


else:
  x_train = x_known_conso.iloc[testperiod:len(x_known_conso)]
  y_train = y_known_conso.iloc[testperiod:len(x_known_conso)]
  x_train['Date'] = pd.to_datetime(x_train['Date'], format='%d/%m/%Y')
  x_train['Date'] = pd.to_datetime(x_train['Date']).astype(int)/100000000000


  x_test = x_known_conso.iloc[0:testperiod]
  y_test = y_known_conso.iloc[0:testperiod]
  x_test['Date'] = pd.to_datetime(x_test['Date'], format='%d/%m/%Y')
  x_test['Date'] = pd.to_datetime(x_test['Date']).astype(int)/100000000000


print(len(x_train), len(x_test), len(y_train), len(y_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWar

960 168 960 168


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#Date are also converted to integers for the models to work
X['Date'] = pd.to_datetime(X['Date'], format='%d/%m/%Y')
X['Date'] = pd.to_datetime(X['Date']).astype(int)/100000000000



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  This is separate from the ipykernel package so we can avoid doing imports until


# **1. K - Neighbors Regressor Model**





In [ ]:
from sklearn.metrics import classification_report

knn_model = KNeighborsRegressor()
knn_model.fit(x_train, y_train)
knn_prediction_test = knn_model.predict(x_test)
print("Accuracy: %.3f (%.3f)" % (knn_prediction_test.mean(), knn_prediction_test.std()))

Accuracy: 20817.685 (4142.380)


In [ ]:
# --------------------- RMSE values for KNN---------------------
RMSE_knn = sqrt(mean_squared_error(y_test,knn_prediction_test))
print("RMSE for K-Nearest Neighbour : {}".format(RMSE_knn))

RMSE for K-Nearest Neighbour : 4057.7231754191084


In [ ]:
# --------------------- Whole Dataset Prediction ---------------
knn_prediction = knn_model.predict(X)
knn_prediction

array([18834.4, 17240.6, 17801. , ..., 21511.4, 19981.6, 21511.4])

# **2. Lasso Model Prediction**


In [ ]:
params = {"alpha": [0.07, 0.2, 0.4, 0.6, 0.9, 2, 5, 10, 15]}
Lasso_model = Lasso(alpha=0.1)
print("lasso_model :",Lasso_model.fit(x_train,y_train))
Lasso_model.fit(x_train, y_train)
Lasso_Prediction_test = Lasso_model.predict(x_test)

lasso_model : Lasso(alpha=0.1)


In [ ]:
RMSE_lasso = sqrt(mean_squared_error(y_test,Lasso_Prediction_test))
print("Lasso Model RMSE Error.{}\n ".format(RMSE_lasso))
# print(x_test)      #------------ Test set for available conso values
# print(X)           # ----------- Whole data set

Lasso Model RMSE Error.3754.7257241358893
 


In [ ]:
# ------------ Predicting for all the data( X)--------------------
#pd.set_option("display.max_rows", 10, "display.max_columns", None)
lasso_prediction = Lasso_model.predict(X)
print(lasso_prediction)

[18206.53961095 15221.49732204 14391.27845007 ... 20633.78403216
 17594.21299118 19401.3237619 ]


# **3. Random Forest Model**

In [ ]:
Rforest_model = RandomForestRegressor(n_estimators=300, n_jobs=-1)
print("model.fit :", Rforest_model.fit(x_train,y_train))
Rforest_model.fit(x_train, y_train)
Rforest_prediction_test = Rforest_model.predict(x_test)

model.fit : RandomForestRegressor(n_estimators=300, n_jobs=-1)


In [ ]:
#print(Rforest_prediction_test)
RMSE_Rforest = sqrt(mean_squared_error(y_test,Rforest_prediction_test))
print("RMSE error for Random Forest Model : {}".format(RMSE_Rforest))

RMSE error for Random Forest Model : 2493.928570727771


In [ ]:
# ------------Prediction for the whole dataset----------
rforest_prediction = Rforest_model.predict(X)
print(rforest_prediction)

[18188.78666667 18148.30333333 18123.7        ... 18326.29333333
 18730.43       18150.58666667]


# **4. XGBoost Model**

In [ ]:
print("XGBOOST")
XGB_model = XGBRegressor()
XGB_model.fit(x_train, y_train)
XGB_prediction_test = XGB_model.predict(x_test)

XGBOOST
[07:34:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
RMSE_xgboost = sqrt(mean_squared_error(y_test,XGB_prediction_test))
print("RMSE error for XGboost Model : {}".format(RMSE_xgboost))

RMSE error for XGboost Model : 1871.5998743540426


In [ ]:
# --------------- Prediction for the whole datset----------------
XGB_prediction = XGB_model.predict(X)
XGB_prediction

array([16855.428, 17666.553, 17651.324, ..., 19340.11 , 18920.092,
       19115.719], dtype=float32)

#**5. Support Vector Machine (SVM)**

In [ ]:
print("SVM_model")
param_grid = {'C': [1000, 10000,50000,100000],'gamma':[1e-8],'kernel': ['rbf']}
svm_model = GridSearchCV(SVR(), param_grid, refit=True, verbose=3)
svm_model.fit(x_train, y_train)
svm_prediction_test = svm_model.predict(x_test)

SVM_model
Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5] END ..C=1000, gamma=1e-08, kernel=rbf;, score=-0.335 total time=   0.1s
[CV 2/5] END ..C=1000, gamma=1e-08, kernel=rbf;, score=-0.267 total time=   0.1s
[CV 3/5] END ...C=1000, gamma=1e-08, kernel=rbf;, score=0.339 total time=   0.1s
[CV 4/5] END ...C=1000, gamma=1e-08, kernel=rbf;, score=0.507 total time=   0.1s
[CV 5/5] END ...C=1000, gamma=1e-08, kernel=rbf;, score=0.520 total time=   0.1s
[CV 1/5] END .C=10000, gamma=1e-08, kernel=rbf;, score=-0.357 total time=   0.1s
[CV 2/5] END .C=10000, gamma=1e-08, kernel=rbf;, score=-0.043 total time=   0.1s
[CV 3/5] END ..C=10000, gamma=1e-08, kernel=rbf;, score=0.368 total time=   0.1s
[CV 4/5] END ..C=10000, gamma=1e-08, kernel=rbf;, score=0.413 total time=   0.1s
[CV 5/5] END ..C=10000, gamma=1e-08, kernel=rbf;, score=0.426 total time=   0.1s
[CV 1/5] END .C=50000, gamma=1e-08, kernel=rbf;, score=-0.550 total time=   0.1s
[CV 2/5] END ..C=50000, gamma=1e-08, ke

In [ ]:
#--------------RMSE Error-------------
RMSE_svm = sqrt(mean_squared_error(svm_prediction_test, y_test))
print("RMSE Error of SVM model : {}".format(RMSE_svm))

RMSE Error of SVM model : 5393.48469580914


In [ ]:
#----------Traning the whole dataset--------
svm_prediction = svm_model.predict(X)
svm_prediction

array([13079.38893657, 16569.36662429, 17471.37010995, ...,
       18487.93446778, 17362.95906391, 18216.02087798])

# **6. Multivariate Adaptive Regression Splines(MARS)**





In [ ]:
!pip install git+https://github.com/scikit-learn-contrib/py-earth@v0.2dev

  Cloning https://github.com/scikit-learn-contrib/py-earth (to revision v0.2dev) to /tmp/pip-req-build-mklux8bk
  Running command git clone -q https://github.com/scikit-learn-contrib/py-earth /tmp/pip-req-build-mklux8bk
  Running command git checkout -b v0.2dev --track origin/v0.2dev
  Switched to a new branch 'v0.2dev'
  Branch 'v0.2dev' set up to track remote branch 'v0.2dev' from 'origin'.


In [ ]:
#import planets.mars
from pyearth import Earth
print("mars_model")
params = {'max_terms':[2, 5, 8, 10, 12, 14], 'max_degree': [1, 2, 3, 4]}
mars_model = GridSearchCV(estimator=Earth(), param_grid=params, cv=3, n_jobs=-1)
mars_model.fit(x_train,y_train)
mars_prediction_test = mars_model.predict(x_test)


mars_model


In [ ]:
#------------------RMSE Error for MARS model ------------
RMSE_mars = sqrt(mean_squared_error(y_test, mars_prediction_test))
print("RMSE error for MARS Model :{}".format(RMSE_mars))

RMSE error for MARS Model :2537.234193713484


In [ ]:
#------------------Prediction for the whole dataset----------------
mars_prediction = mars_model.predict(X)
mars_prediction

array([16515.6271415 , 16902.71346564, 16867.65243994, ...,
       19332.73967715, 16350.73297444, 17339.796154  ])

# Prediction comparison

In [ ]:
#-------------------Adding all the predicted Conso Values values of the models into table-----------------

data['Rforest_prediction'] = rforest_prediction     # 1. Random forest
data['Lasso_Prediction'] = lasso_prediction         # 2. Lasso
data['KNN_Prediction'] = knn_prediction             # 3. KNN
data['XGB_prediction'] = XGB_prediction             # 4. XGB
data['SVM_prediction'] = svm_prediction             # 5. SVM
data['MARS_prediction'] = mars_prediction           # 6. MARS

output = data.drop(['Holiday','Bridge','temperature','Day','variable 2'], axis = 1)
print(output)

            Date  Heure    Conso  Rforest_prediction  Lasso_Prediction  \
0     01/10/2022      0  14889.0        18188.786667      18206.539611   
1     01/10/2022      1  14812.0        18148.303333      15221.497322   
2     01/10/2022      2  14469.0        18123.700000      14391.278450   
3     01/10/2022      3  14670.0        18051.190000      12415.955785   
4     01/10/2022      4  14834.0        18393.500000      10219.193187   
...          ...    ...      ...                 ...               ...   
1291  23/11/2022     19      NaN        20794.076667      24982.755247   
1292  23/11/2022     20      NaN        20410.350000      23800.094441   
1293  23/11/2022     21      NaN        18326.293333      20633.784032   
1294  23/11/2022     22      NaN        18730.430000      17594.212991   
1295  23/11/2022     23      NaN        18150.586667      19401.323762   

      KNN_Prediction  XGB_prediction  SVM_prediction  MARS_prediction  
0            18834.4    16855.427734   

In [ ]:
# Errors for the test period

print("Lasso               : {}".format(RMSE_lasso))
print("K-Nearest Neighbour : {}".format(RMSE_knn))
print("Rnadom Forest       : {}".format(RMSE_Rforest))
print("xgboost             : {}".format(RMSE_xgboost))
print("SVM                 : {}".format(RMSE_svm))
print("mars                : {}".format(RMSE_mars))

Lasso               : 3754.7257241358893
K-Nearest Neighbour : 4057.7231754191084
Rnadom Forest       : 2493.928570727771
xgboost             : 1871.5998743540426
SVM                 : 5393.48469580914
mars                : 2537.234193713484 
